# Assignment 4
## Group 1: Tara Bode and Hankun Li

## Part A: Text-based Industry Classification

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

### Functions

In [2]:
def getHeaderVariables(filing):
    CIK = filing[ filing.find('CENTRAL INDEX KEY:') + 18  : filing.find('STANDARD') ]
    CIK = ''.join(CIK.split())
    #
    CCN = filing[ filing.find('COMPANY CONFORMED NAME:') + 23 : filing.find('CENTRAL INDEX KEY') ]
    CCN = CCN.rstrip().lstrip()
    #
    CST = filing[ filing.find('CONFORMED SUBMISSION TYPE:') + 26 : filing.find('PUBLIC DOCUMENT COUNT') ]
    CST = ''.join(CST.split())
    #
    CPR = filing[ filing.find('CONFORMED PERIOD OF REPORT:') + 27 : filing.find('FILED AS OF DATE') ]
    CPR = ''.join(CPR.split())
    #
    Length = 'skip'
    
    sec_info = '{} | {} | {} | {} | {}\n'.format(CIK, CCN, CST, CPR, Length)
    return (sec_info)


In [3]:
def cleanHTML(raw_html):
    cleanr = re.compile('<.*?>', flags=re.DOTALL)
    cleantext = re.sub(cleanr, '', raw_html)
    return replace_entities(cleantext)

In [4]:
def getBusinessSection(filing):
    # USE REGEXR
    BDS = filing[ filing.find('Item 1. ') : filing.find('Item 1A. ') ]
    #print('length_business_section',len(BDS))
    #print(BDS)
    return BDS
    #

### Loop in A3 - Successfully Inherited 

In [5]:
mydirectory = '/blue/acg7849/share/10Ks'
file_list = glob.glob(mydirectory + '/*.txt')
#print(file_list)

In [6]:
# write header variables and length of business section to disk
with open (r'/blue/acg7849/hli1/SEC_Information.csv', 'w') as f:
    f.write('File Name | Central Index Key | Company Name | Submission Type | Period of Report | Length of Business Description\n')

for file_path in file_list[0:10]: # use 10 files to test
    # read the file
    with open(file_path) as f:
        filing = f.read()
        
    # only use until /DOCUMENT
    first_section = filing[0: filing.find('</DOCUMENT')]
    
    # get header variables
    headervars = str(getHeaderVariables(first_section))
    
    # isolate business section
    business_section = str(getBusinessSection(first_section))
    
    #need to input FileName
    shortName = file_path[file_path.rfind("/")+1:]
    
    # write business section to disk       
    with open(r'/blue/acg7849/hli1/item1/'+ shortName, 'w', encoding="utf-8") as f:
        f.write(cleanHTML(business_section))
   
    with open (r'/blue/acg7849/hli1/SEC_Information.csv', 'a', encoding="utf-8") as f:
        f.write(shortName + ' | ' + cleanHTML(headervars))

    #with open (r'/blue/acg7849/tbode/SEC_Information.csv', 'a', encoding="utf-8") as f:
     #   f.write(shortName + ' | ' + cleanHTML(headervars) + ' | ' + len(cleanHTML(business_section)))


 ### Recommended - Use Summary Files

In [70]:
# matching filenames

with open ('/blue/acg7849/share/BS/summary.txt') as a: 
    summary = a. readlines()
print (summary[0:5])

['CIK|coname|formtype|date|filename|length\n', '0001289850|NeuroMetrix, Inc.|10-K|20171231|267762.txt|132616\n', '0001345016|YELP INC|10-K|20191231|280603.txt|152323\n', '0001245791|GI DYNAMICS, INC.|10-K|20171231|267435.txt|28236\n', '0001772177|KURA SUSHI USA, INC.|10-K|20190831|275323.txt|38468\n']


In [13]:
# matching filenames
with open ('/blue/acg7849/share/BS/summary.txt') as a: 
    summary = a. readlines()
print (summary[0:5])

sumdf = pd.DataFrame (summary)
#sumdf.str.split(pat="|")


new = sumdf[0].str.split("|", expand = True)
sumdf["CIK"]= new[0]
sumdf["coName"]= new[1]
sumdf["formtype"]= new[2]
sumdf["date"]= new[3]
sumdf["fName"]= new[4]
sumdf["length"]= new[5]
sumdf = sumdf.drop(columns=[0])
sumdf = sumdf.drop([0])

print (sumdf[0:5])                        
#Initialize csv - no clusters
sumdf.to_csv('ASG4.csv', index = True)

print('CSV downloaded as ASG4.csv')

CSV downloaded as ASG4.csv


### New Loop for Clustering

In [7]:
newdirectory = '/blue/acg7849/share/BS/item1'
file_list1 = glob.glob(newdirectory + '/*.txt')
#print(file_list1) # success!

In [8]:
obj = pd.DataFrame(file_list)
obj.to_csv('file_list.csv', index = True) # csv preapared to use

In [9]:
badFiles_raw = []

In [10]:
for i in file_list[0:5]:
    print (i)
    # read the file
    with open(i) as f:
        filing = f.read()
        
    # grab the length of the file
    print (len(filing))

    # Ignore business section files with less than 1000 characters.
    if len(filing) < 50000: # test. should be 1000 here
        print ('bad')
        badFiles_raw.append (i)
    # should end this file and return to next one
    # No. of cluster should be 0 or NA

    
    #need to input FileName
    shortName = i[i.rfind("/")+1:]
    print(shortName)
    # match shortnames to cluster No.
    
    # tokenize
    #word_tokenize (filing[0:10]) # bit different from split - include punctuations
#print(badFiles_raw)

/blue/acg7849/share/10Ks/267762.txt
7858323
267762.txt
/blue/acg7849/share/10Ks/280603.txt
15640951
280603.txt
/blue/acg7849/share/10Ks/267435.txt
9945879
267435.txt
/blue/acg7849/share/10Ks/275323.txt
3221064
275323.txt
/blue/acg7849/share/10Ks/277539.txt
15953715
277539.txt


In [11]:
print(badFiles_raw)

[]


In [12]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [13]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [14]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [15]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Make a List of Full Texts to do K-Means
reference: http://brandonrose.org/clustering

In [90]:
totalvocab_stemmed = []
totalvocab_tokenized = []

newdirectory = '/blue/acg7849/share/BS/item1'
file_list1 = glob.glob(newdirectory + '/*.txt')
#print(file_list1) # success!

filing1 = []
n=-1

for i in file_list1[0:50]:
    print (i)
    # read the file
    with open(i) as f:
        #filing = f.read()
        filing = f.readlines() # some were one line and some were multi lines...so need combine
        print("num of elem:"len(filing))
        print(n)
        n = n +1
        if len(filing) > 1:
            print(n) # this is the (n-1)th file
            filing[n : 99999] = [''.join(filing[n : 99999])]
            #print("this is",filing[-1]) # seems that the last element is involved
        filing1 += filing
print(len(filing1))  # there can be a big number so the last one is not looping

/blue/acg7849/share/BS/item1/267762.txt
1
-1
/blue/acg7849/share/BS/item1/280603.txt
1
0
/blue/acg7849/share/BS/item1/267435.txt
489
1
2
/blue/acg7849/share/BS/item1/275323.txt
176
2
3
/blue/acg7849/share/BS/item1/277539.txt
1
3
/blue/acg7849/share/BS/item1/270762.txt
2035
4
5
/blue/acg7849/share/BS/item1/274077.txt
9
5
6
/blue/acg7849/share/BS/item1/265478.txt
1
6
/blue/acg7849/share/BS/item1/280595.txt
1685
7
8
/blue/acg7849/share/BS/item1/283125.txt
11
8
9
/blue/acg7849/share/BS/item1/281916.txt
4
9
10
/blue/acg7849/share/BS/item1/282859.txt
2121
10
11
/blue/acg7849/share/BS/item1/284462.txt
1
11
/blue/acg7849/share/BS/item1/270024.txt
1
12
/blue/acg7849/share/BS/item1/280098.txt
747
13
14
/blue/acg7849/share/BS/item1/280672.txt
3
14
15
/blue/acg7849/share/BS/item1/274227.txt
2
15
16
/blue/acg7849/share/BS/item1/266806.txt
1
16
/blue/acg7849/share/BS/item1/283268.txt
1
17
/blue/acg7849/share/BS/item1/266373.txt
649
18
19
/blue/acg7849/share/BS/item1/269026.txt
1042
19
20
/blue/acg78

In [17]:
type(filing1)

list

In [75]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in filing1:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'filing1', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [76]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 72841 items in vocab_frame


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(filing1) #fit the vectorizer to a iterable list of txt

print(tfidf_matrix.shape)

/home/hli1/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


CPU times: user 1.77 s, sys: 4.22 ms, total: 1.78 s
Wall time: 1.79 s
(49, 170)


In [78]:
terms = tfidf_vectorizer.get_feature_names()

In [79]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [80]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# KMC

In [81]:
from sklearn.cluster import KMeans

num_clusters = 5                            # doc num > num of clst

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 62 ms, sys: 0 ns, total: 62 ms
Wall time: 63.6 ms


In [82]:
import joblib
#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle
joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [83]:
Comps = { 'shortName': shortName, 'cluster': clusters}
frame = pd.DataFrame(Comps, index = [clusters] , columns = ['shortName', 'cluster'])

In [84]:
print(frame) # why do all the short names same? seem improper

    shortName  cluster
1  277539.txt        1
1  277539.txt        1
2  277539.txt        2
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
3  277539.txt        3
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
1  277539.txt        1
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
2  277539.txt        2
3  277539.txt        3
0  277539.txt        0
0  277539.txt        0
2  277539.txt        2
0  277539.txt        0
0  277539.txt        0
0  277539.txt        0
1  277539.txt        1
0  277539.txt        0
4  277539.txt        4
4  277539.txt        4
4  277539.txt        4
4  277539.txt        4
4  277539.t

In [85]:
frame['cluster'].value_counts()

0    24
4    10
1     8
2     5
3     2
Name: cluster, dtype: int64

In [86]:
# which cluster are the comps in accordingly?

## Part B: Sentiment

In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


### Setup

In [2]:
import re
import pandas as pd
import time # needed for time.sleep
from selenium import webdriver

# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# new driver (opens browser window)
driver = webdriver.Chrome(r'/Users/Tara/Desktop/Summer A 2021/ACG7849- Web Crawling/selenium_files/chromedriver')
driver.get("https://www.fandango.com/movie-reviews")

In [3]:
from gensim.parsing.preprocessing import remove_stopwords

import glob, re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.collocations import *

import string
stopWords = set(stopwords.words('english') )

#digitsList = ['1','2','3','4','5','6','7','8','9','0']

/Users/Tara/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to /Users/Tara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Tara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
el = driver.find_element_by_tag_name("a")
elements = driver.find_elements_by_tag_name("a")
pagesToCrawl = []
for el in elements[0:331]:
    parent = el.find_element_by_xpath('..') 
    if el.get_attribute("class")=="cta mrp__table-movie-reviews-link":      
        pagesToCrawl.append(el.get_attribute("href")  )

### Functions

In [5]:
def sentimentScores(text):
  
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
  
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary- contains pos, neg, neu, and compound scores
    sentiment_dict = sid_obj.polarity_scores(text)
      
    print('Reviews rated as ', sentiment_dict['neg']*100, '% Negative')
    print('Reviews rated as ', sentiment_dict['neu']*100, '% Neutral')
    print('Reviews rated as ', sentiment_dict['pos']*100, '% Positive')
  
    print('Movie Overall Rated As', end = ' ')
  
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.5 :
        print('Positive! YAY 😄')
  
    elif sentiment_dict['compound'] <= - 0.5 :
        print('Negative! BOO 😢')
  
    else :
        print('Neutral 😐')

### Takes Sentiment of All Individual Reviews w/out Necessary Removals (to demonstrate difference)

In [6]:
# Correct Version- takes 1 sentiment of all reviews, no removal
with open (r'/Users/Tara/Desktop/webcrawl.txt', 'w') as f:
    f.write('Movie Name | Critic Score | Audience Score | Review \n')

movie_list = []
critic_score_list = []
audience_score_list = []
review_messages = []

for item in range(len(pagesToCrawl)):
#for item in range(len(pagesToCrawl[0:15])):    
    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        movie_list= driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name');
        critic_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        print('\n Movie Name: {} | '.format(movie_list) + 'Critic Score: {} | '.format(critic_score_list) + 'Audience Score: {}\n'.format(audience_score_list))
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            review_messages.append(text)
        sentimentScores(review_messages)

    except:
        print('\nNo rating score to display for '+movie_list+' . . . \n')


 Movie Name: A Quiet Place Part II (2021) | Critic Score: 91% | Audience Score: 93%

Reviews rated as  4.8 % Negative
Reviews rated as  63.5 % Neutral
Reviews rated as  31.7 % Positive
Movie Overall Rated As Positive! YAY 😄

No rating score to display for In the Heights (2021) . . . 


 Movie Name: The Conjuring: The Devil Made Me Do It (2021) | Critic Score: 59% | Audience Score: 85%

Reviews rated as  7.5 % Negative
Reviews rated as  65.60000000000001 % Neutral
Reviews rated as  26.900000000000002 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Cruella (2021) | Critic Score: 74% | Audience Score: 97%

Reviews rated as  8.200000000000001 % Negative
Reviews rated as  62.6 % Neutral
Reviews rated as  29.2 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Spirit Untamed (2021) | Critic Score: 43% | Audience Score: 97%

Reviews rated as  6.3 % Negative
Reviews rated as  61.7 % Neutral
Reviews rated as  31.900000000000002 % Positive
Movie Overall Rated As Pos


No rating score to display for A Father's Legacy . . . 


 Movie Name: Portrait of a Lady on Fire | Critic Score: 98% | Audience Score: 92%

Reviews rated as  8.4 % Negative
Reviews rated as  67.80000000000001 % Neutral
Reviews rated as  23.799999999999997 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Mortal Kombat (2021) | Critic Score: 55% | Audience Score: 86%

Reviews rated as  8.7 % Negative
Reviews rated as  67.80000000000001 % Neutral
Reviews rated as  23.400000000000002 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Minions (2015) | Critic Score: 55% | Audience Score: 49%

Reviews rated as  8.7 % Negative
Reviews rated as  68.0 % Neutral
Reviews rated as  23.400000000000002 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: The Boss Baby | Critic Score: 53% | Audience Score: 51%

Reviews rated as  8.6 % Negative
Reviews rated as  68.10000000000001 % Neutral
Reviews rated as  23.3 % Positive
Movie Overall Rated As Positive! YAY 😄


### Takes Sentiment of All Individual Reviews w/ Necessary Removals 

In [7]:
# Correct Version- takes 1 sentiment of all reviews removing stopwords, punctuation, and words with starting digits
with open (r'/Users/Tara/Desktop/webcrawl.txt', 'w') as f:
    f.write('Movie Name | Critic Score | Audience Score | Review \n')

movie_list = []
critic_score_list = []
audience_score_list = []
review_messages = []

#for item in range(len(pagesToCrawl)):
for item in range(len(pagesToCrawl[1:25])):    
    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        movie_list= driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name');
        critic_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        print('\n Movie Name: {} | '.format(movie_list) + 'Critic Score: {} | '.format(critic_score_list) + 'Audience Score: {}\n'.format(audience_score_list))
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            filtered_text = "".join(filter(lambda x:x[0] != x.isdigit(), text))
            filtered_text = filtered_text.translate(str.maketrans('', '', string.punctuation))
            filtered_text = remove_stopwords(filtered_text)  
            review_messages.append(filtered_text)
        sentimentScores(review_messages)

    except:
        print('\nNo rating score to display for '+movie_list+' . . . \n')


 Movie Name: A Quiet Place Part II (2021) | Critic Score: 91% | Audience Score: 93%

Reviews rated as  8.4 % Negative
Reviews rated as  45.5 % Neutral
Reviews rated as  46.1 % Positive
Movie Overall Rated As Positive! YAY 😄

No rating score to display for In the Heights (2021) . . . 


 Movie Name: The Conjuring: The Devil Made Me Do It (2021) | Critic Score: 59% | Audience Score: 85%

Reviews rated as  12.2 % Negative
Reviews rated as  52.6 % Neutral
Reviews rated as  35.199999999999996 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Cruella (2021) | Critic Score: 74% | Audience Score: 97%

Reviews rated as  12.9 % Negative
Reviews rated as  48.199999999999996 % Neutral
Reviews rated as  38.9 % Positive
Movie Overall Rated As Positive! YAY 😄

 Movie Name: Spirit Untamed (2021) | Critic Score: 43% | Audience Score: 97%

Reviews rated as  10.299999999999999 % Negative
Reviews rated as  46.800000000000004 % Neutral
Reviews rated as  42.9 % Positive
Movie Overall Rated As

### Observations
Most movies are rated as "Positive" because the movies have all been quite highly rated by the audience, so the output seems to match Fandango's movie reviews!

### Iterating over Use of Functions

In [8]:
def navigateToWebsite():
    el = driver.find_element_by_tag_name("a")
    elements = driver.find_elements_by_tag_name("a")
    pagesToCrawl = []
    for el in elements[0:331]:
        parent = el.find_element_by_xpath('..') 
        if el.get_attribute("class")=="cta mrp__table-movie-reviews-link":      
            pagesToCrawl.append(el.get_attribute("href")  )
            return(pagesToCrawl)
    
def getContents(item):
    movie_list = [] 
    critic_score_list = []
    audience_score_list = []

    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        movie_list= driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name');
        critic_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        #return('\n Movie Name: {} | '.format(movie_list) + 'Critic Score: {} | '.format(critic_score_list) + 'Audience Score: {}\n'.format(audience_score_list))
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            filtered_text = "".join(filter(lambda x:x[0] != x.isdigit(), text))
            filtered_text = filtered_text.translate(str.maketrans('', '', string.punctuation))
            filtered_text = remove_stopwords(filtered_text)  
            review_messages.append(filtered_text)
        #return(sentimentScores(review_messages)  )
        movie_info =  ('\n {} | {} | {} | {}\n'.format(movie_list, critic_score_list, audience_score_list, sentimentScores(review_messages)))
        print(movie_info)       
        #return('\n {} | {} | {} | {}\n'.format(movie_list, critic_score_list, audience_score_list, sentimentScores(review_messages)))


    except:
        print('No rating score to display for '+pagesToCrawl[item]+' . . . \n')
        
def sentimentScores(text):
  
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
  
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary- contains pos, neg, neu, and compound scores
    sentiment_dict = sid_obj.polarity_scores(text)
        
    print('Reviews rated as ', sentiment_dict['neg']*100, '% Negative')
    print('Reviews rated as ', sentiment_dict['neu']*100, '% Neutral')
    print('Reviews rated as ', sentiment_dict['pos']*100, '% Positive')
  
    print('Movie Overall Rated As', end = ' ')
  
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print('Positive')
  
    elif sentiment_dict['compound'] <= - 0.05 :
        print('Negative')
  
    else :
        print('Neutral')

In [9]:
with open (r'/Users/Tara/Desktop/webcrawl.txt', 'w') as f:
    f.write('Movie Name | Critic Score | Audience Score | Sentiment Analysis \n')

for item in range(len(pagesToCrawl[0:1])):

    movieReviews = str(getContents(item))
    
    # update summary
    with open(r'/Users/Tara/Desktop/webcrawl.txt', 'a', encoding="utf-8") as f:
        #f.write('\n{} | {} | {} | {}\n'.format( movie_list, critic_score_list, audience_score_list, sentimentScores(review_messages) ))
        f.write(movieReviews)

Reviews rated as  13.4 % Negative
Reviews rated as  51.0 % Neutral
Reviews rated as  35.6 % Positive
Movie Overall Rated As Positive

 A Quiet Place Part II (2021) | 91% | 93% | None



## Initial Guide Only


In [ ]:
# Beautiful Version
with open (r'/Users/Tara/Desktop/webcrawl.txt', 'w') as f:
    f.write('Movie Name | Critic Score | Audience Score | Review \n')

movie_list = []
critic_score_list = []
audience_score_list = []


for item in range(len(pagesToCrawl[0:2])):    
    driver.get(pagesToCrawl[item])
    time.sleep(5)
    try:
        movie_list= driver.find_element_by_xpath('//*[@id="subnav"]/div/div/h1/a').get_attribute('data-name');
        critic_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[0].text
        audience_score_list = driver.find_elements_by_class_name('movie-ratings-tabs__rating')[1].text
        #print('{} | {} | {}\n'.format(movie_list,critic_score_list,audience_score_list))
        review_list = driver.find_elements_by_class_name('rt-audience-reviews__content')
        for review in review_list:
            text = review.text
            with open('/Users/Tara/Desktop/webcrawl.txt', 'a', encoding="utf-8") as f:
                f. write('{} | {} | {} | {} \n'.format(movie_list,critic_score_list,audience_score_list, text))

    except:
        print('No rating score to display for '+pagesToCrawl[item]+' . . . \n')